In [38]:
import pandas as pd
import zipfile
import os
import re
import numpy as np
import unicodedata

In [3]:
#Descomprimimos
destino = 'datasets/'
destino2 = 'acceso/'

if not os.path.exists(destino):
    os.makedirs(destino)

with zipfile.ZipFile("univaciada_1.zip", 'r') as archivo_zip:
    archivo_zip.extractall(destino)

with zipfile.ZipFile("univaciada_2.zip", 'r') as archivo_zip:
    archivo_zip.extractall(destino)

with zipfile.ZipFile("univaciada_3.zip", 'r') as archivo_zip:
    archivo_zip.extractall(destino)

with zipfile.ZipFile("INe.zip", 'r') as archivo_zip:
    archivo_zip.extractall()

if not os.path.exists(destino2):
    os.makedirs(destino2)

with zipfile.ZipFile("acceso.zip", 'r') as archivo_zip:
    archivo_zip.extractall(destino2)

In [13]:
#Preprocesamiento
ficheros = os.listdir(destino)
ficheros

#Solo cargamos las columnas necesarias
columnas_interes = ["curso_academico", 'des_municipio_residencia',"cod_tipo_estudio", "cod_municipio_residencia",  "lat_municipio_residencia", "lon_municipio_residencia"]

dataframes = []

#Limpieza de datos, nos quedamos solo con estudiantes de Grado españoles. Añadimos una columna con la universidad del estudiante.
for fichero in ficheros:
    try:
        df = pd.read_csv(destino+"/"+fichero, usecols=columnas_interes, low_memory=False)
        df = df[df["cod_tipo_estudio"] == 'G']  #Estudiantes de Grado
        df = df[df['des_municipio_residencia'] != 'MUNICIPIO EXTRANJERO PERTENECIENTE A LA U.E.']
        df = df[df['des_municipio_residencia'] != 'MUNICIPIO EXTRANJERO NO PERTENECIENTE A LA U.E.']
        df = df.drop(columns=["des_municipio_residencia"])
        df = df.dropna()
        df['cod_municipio_residencia'] = df['cod_municipio_residencia'].astype('int')
        nombre = fichero.split('/')[-1]
        universidad = re.match(r'^([^-]+)', nombre).group(1)
        df['universidad'] = universidad
        dataframes.append(df)  
    except Exception as e:
        print(f"Error al cargar el archivo desde {fichero}: {e}")

#Concatenamos
df_matriculas = pd.concat(dataframes, ignore_index=True)

#Calculamos el tamaño de la burbuja normalizando los datos
df_matriculas['count'] = 1
df_grouped = df_matriculas.groupby(['lat_municipio_residencia', 'lon_municipio_residencia', "cod_municipio_residencia",'universidad', 'curso_academico'], as_index=False).agg({
    'count': 'sum'
})

min_count = df_grouped['count'].min()
max_count = df_grouped['count'].max()
#df_grouped['scaled_count'] = 1 + 19 * (df_grouped['count'] - min_count) / (max_count - min_count)
df_grouped['scaled_count'] = 1.2
df_grouped['log_count'] = df_grouped['count'].apply(lambda x: max(1, x)).apply(np.log)
df_grouped

,lat_municipio_residencia,lon_municipio_residencia,cod_municipio_residencia,universidad,curso_academico,count,scaled_count,log_count
0,27.75379,-18.003640,38013,uam,2017-18,1,1.2,0.000000
1,27.75379,-18.003640,38013,ucm,2017-18,5,1.2,1.609438
2,27.75379,-18.003640,38013,ucm,2018-19,4,1.2,1.386294
3,27.75379,-18.003640,38013,ucm,2019-20,4,1.2,1.386294
4,27.75379,-18.003640,38013,ucm,2020-21,4,1.2,1.386294
...,...,...,...,...,...,...,...,...
46021,43.73261,-7.674875,27064,uam,2022-23,1,1.2,0.000000
46022,43.73261,-7.674875,27064,uc3m,2020-21,1,1.2,0.000000
46023,43.73261,-7.674875,27064,uc3m,2021-22,1,1.2,0.000000
46024,43.73261,-7.674875,27064,uc3m,2022-23,1,1.2,0.000000


In [132]:
ine = pd.read_csv("INe.csv", sep=";", low_memory=False)
#Nos quedamos con la población total del 2017 en adelante
ine = ine[ine["Sexo"] == "Total"]
ine = ine[ine["Periodo"] >= 2017]
ine[['cod_municipio_residencia', 'municipio']] = ine['Municipios'].str.split(' ', n=1, expand=True)
ine['cod_municipio_residencia'] = ine['cod_municipio_residencia'].astype(int)
ine['curso_academico'] = ine['Periodo'].astype(int).apply(lambda x: f"{x}-{(x + 1) % 100:02d}")
ine = ine.dropna()
ine['Total'] = ine['Total'].str.replace('.', '', regex=False).astype(int)
ine['Total'] = ine['Total'].astype('int')
ine['poblacion'] = ine['Total'].apply(
    lambda x: 'Muy densa' if x > 500000 else
              'Densa' if x > 100000 else
              'Moderadamente densa' if x > 25000 else
              'Poco densa' if x > 5000 else
              'Muy poco densa'
)
ine = ine.drop(columns=['Municipios',"Sexo","Periodo", "Total"])
ine

,cod_municipio_residencia,municipio,curso_academico,poblacion
0,44001,Ababuj,2024-25,Muy poco densa
1,44001,Ababuj,2023-24,Muy poco densa
2,44001,Ababuj,2022-23,Muy poco densa
3,44001,Ababuj,2021-22,Muy poco densa
4,44001,Ababuj,2020-21,Muy poco densa
...,...,...,...,...
707922,4103,Zurgena,2021-22,Muy poco densa
707923,4103,Zurgena,2020-21,Muy poco densa
707924,4103,Zurgena,2019-20,Muy poco densa
707925,4103,Zurgena,2018-19,Muy poco densa


In [15]:
#Juntamos los conjuntos
df_merged = df_grouped.merge(ine, on=['cod_municipio_residencia', 'curso_academico'], how='left')
df_merged

,lat_municipio_residencia,lon_municipio_residencia,cod_municipio_residencia,universidad,curso_academico,count,scaled_count,log_count,municipio,poblacion
0,27.75379,-18.003640,38013,uam,2017-18,1,1.2,0.000000,Frontera,Muy poco densa
1,27.75379,-18.003640,38013,ucm,2017-18,5,1.2,1.609438,Frontera,Muy poco densa
2,27.75379,-18.003640,38013,ucm,2018-19,4,1.2,1.386294,Frontera,Muy poco densa
3,27.75379,-18.003640,38013,ucm,2019-20,4,1.2,1.386294,Frontera,Muy poco densa
4,27.75379,-18.003640,38013,ucm,2020-21,4,1.2,1.386294,Frontera,Muy poco densa
...,...,...,...,...,...,...,...,...,...,...
46021,43.73261,-7.674875,27064,uam,2022-23,1,1.2,0.000000,"Vicedo, O",Muy poco densa
46022,43.73261,-7.674875,27064,uc3m,2020-21,1,1.2,0.000000,"Vicedo, O",Muy poco densa
46023,43.73261,-7.674875,27064,uc3m,2021-22,1,1.2,0.000000,"Vicedo, O",Muy poco densa
46024,43.73261,-7.674875,27064,uc3m,2022-23,1,1.2,0.000000,"Vicedo, O",Muy poco densa


In [16]:
rows_with_nan = ine[ine.isna().any(axis=1)]

print("Filas con valores NaN:")
print(rows_with_nan)

Filas con valores NaN:
Empty DataFrame
Columns: [cod_municipio_residencia, municipio, curso_academico, poblacion]
Index: []


In [17]:
df_merged.to_csv('matricula.csv', index=False)  # index=False para no guardar el índice

In [4]:
##ACCESO

In [130]:
pd.set_option("display.max_rows", 20)
#Preprocesamiento
ficheros = os.listdir(destino2)
ficheros

#Solo cargamos las columnas necesarias
columnas_interes = ["curso_academico", 'des_titulacion',"des_provincia_centro_sec","cod_municipio_centro_sec","nota_admision"]

dataframes = []

#Limpieza de datos, nos quedamos solo con estudiantes de Grado españoles. Añadimos una columna con la universidad del estudiante.
for fichero in ficheros:
    try:
        df = pd.read_csv(destino2+"/"+fichero, usecols=columnas_interes, low_memory=False)   
        df = df.dropna()
        dataframes.append(df)  
    except Exception as e:
        print(f"Error al cargar el archivo desde {fichero}: {e}")

#Concatenamos
df_acceso = pd.concat(dataframes, ignore_index=True)

#Unificamos la descripción de las titulaciones
#Letras en minúscula
df_acceso['des_titulacion'] = df_acceso['des_titulacion'].str.lower()
df_acceso['cod_municipio_centro_sec'] = df_acceso['cod_municipio_centro_sec'].astype('int')
df_acceso['nota_admision'] = df_acceso['nota_admision'].str.replace(',', '.').astype(float)

#Quitamos las tildes para normalizar
def quitar_tildes(texto):
    if isinstance(texto, str):  # Asegurarse de que sea un string
        texto_normalizado = unicodedata.normalize('NFD', texto)
        texto_sin_tildes = ''.join(c for c in texto_normalizado if unicodedata.category(c) != 'Mn')
        return texto_sin_tildes
    return texto  # Si no es string, devolver como está
#Quitamos los dobles grados, ya aparecen los individuales
df_acceso = df_acceso[~df_acceso['des_titulacion'].str.contains('doble', case=False, na=False)]
df_acceso = df_acceso[~df_acceso['des_titulacion'].str.contains('triple', case=False, na=False)]
df_acceso = df_acceso[~df_acceso['des_titulacion'].str.contains('/', case=False, na=False)]
df_acceso = df_acceso[~df_acceso['des_titulacion'].str.contains('programa de estudios conjuntos ', case=False, na=False)]
df_acceso = df_acceso[~df_acceso['des_titulacion'].str.contains('programa de estudios conjunto ', case=False, na=False)]
df_acceso = df_acceso[~df_acceso['des_titulacion'].str.contains('programa conjunto ', case=False, na=False)]
df_acceso = df_acceso[~df_acceso['des_titulacion'].str.contains('programa de estudio conjunto ', case=False, na=False)]

# Aplicar la función a la columna
df_acceso['des_titulacion'] = df_acceso['des_titulacion'].apply(quitar_tildes)
df_acceso['des_titulacion'] = df_acceso['des_titulacion'].str.strip().str.lower()
#Quitamos el "por la Universidad..."
df_acceso['des_titulacion'] = df_acceso['des_titulacion'].str.replace(r' por.*', '', regex=True)
#Quitamos 
df_acceso['des_titulacion'] = df_acceso['des_titulacion'].str.replace(r'^graduado o graduada en ', '', regex=True)
df_acceso['des_titulacion'] = df_acceso['des_titulacion'].str.replace(r'^grado en ', '', regex=True)
df_acceso['des_titulacion'] = df_acceso['des_titulacion'].str.replace(r'^degree in ', '', regex=True)
df_acceso['des_titulacion'] = df_acceso['des_titulacion'].str.replace(r'^graduado/a en ', '', regex=True)
df_acceso['des_titulacion'] = df_acceso['des_titulacion'].str.replace(r'^maestro en ', '', regex=True)
df_acceso['des_titulacion'] = df_acceso['des_titulacion'].str.replace(r'^maestro de ', '', regex=True)
df_acceso['des_titulacion'] = df_acceso['des_titulacion'].str.replace(r'o/a', 'o', regex=True)
df_acceso['des_titulacion'] = df_acceso['des_titulacion'].str.replace(r'grado abierto uc3m en', '', regex=True)
df_acceso['des_titulacion'] = df_acceso['des_titulacion'].str.replace(r' de segovia', '', regex=True)
df_acceso['des_titulacion'] = df_acceso['des_titulacion'].str.replace(r'\(.*?\)', '', regex=True)
df_acceso['des_titulacion'] = df_acceso['des_titulacion'].str.split(' centro').str[0]
df_acceso['des_titulacion'] = df_acceso['des_titulacion'].str.split(' instituto').str[0]
df_acceso['des_titulacion'] = df_acceso['des_titulacion'].str.replace(r' i$', '', regex=True)

df_acceso['des_titulacion'] = df_acceso['des_titulacion'].str.strip().str.lower()
df_acceso

,curso_academico,des_titulacion,des_provincia_centro_sec,cod_municipio_centro_sec,nota_admision
0,2017-18,estudios internacionales,Córdoba,14038,12.130
1,2017-18,estudios internacionales,Madrid,28079,11.299
2,2017-18,estudios internacionales,Madrid,28079,10.649
3,2017-18,estudios internacionales,Salamanca,37274,10.406
4,2017-18,estudios internacionales,Madrid,28079,13.271
...,...,...,...,...,...
239281,2023-24,enfermeria,Valladolid,47186,7.210
239282,2023-24,enfermeria,Palencia,34120,12.678
239283,2023-24,enfermeria,Santa Cruz de Tenerife,38038,11.067
239284,2023-24,enfermeria,Valladolid,47186,11.846


In [127]:
#Cuantas titulaciones diferentes quedan?
pd.set_option("display.max_rows",20)
pd.set_option("display.max_colwidth", None)
distinct_values = df_acceso['des_titulacion'].drop_duplicates()

# Convertir los valores únicos en un DataFrame
distinct_df = pd.DataFrame(distinct_values, columns=['des_titulacion'])
distinct_df

,des_titulacion
0,estudios internacionales
43,enfermeria
137,quimica
182,antropologia social y cultural
194,ciencias de la alimentacion
...,...
223973,estudios clasicos
224139,historia y ciencias de la musica
228891,ingenieria energetica
232207,geografia y planificacion territorial


In [133]:
#uno con los datos del INe para saber si el centro de origen pertenece a un municipio muy poblado...
ine = ine.rename(columns={'cod_municipio_residencia': 'cod_municipio_centro_sec'})
df_merged2 = df_acceso.merge(ine, on=['cod_municipio_centro_sec', 'curso_academico'], how='left')
df_merged2

,curso_academico,des_titulacion,des_provincia_centro_sec,cod_municipio_centro_sec,nota_admision,municipio,poblacion
0,2017-18,estudios internacionales,Córdoba,14038,12.130,Lucena,Moderadamente densa
1,2017-18,estudios internacionales,Madrid,28079,11.299,Madrid,Muy densa
2,2017-18,estudios internacionales,Madrid,28079,10.649,Madrid,Muy densa
3,2017-18,estudios internacionales,Salamanca,37274,10.406,Salamanca,Densa
4,2017-18,estudios internacionales,Madrid,28079,13.271,Madrid,Muy densa
...,...,...,...,...,...,...,...
214984,2023-24,enfermeria,Valladolid,47186,7.210,Valladolid,Densa
214985,2023-24,enfermeria,Palencia,34120,12.678,Palencia,Moderadamente densa
214986,2023-24,enfermeria,Santa Cruz de Tenerife,38038,11.067,Santa Cruz de Tenerife,Densa
214987,2023-24,enfermeria,Valladolid,47186,11.846,Valladolid,Densa


In [135]:
df_merged2.to_csv('acceso.csv', index=False)  # index=False para no guardar el índice